In [10]:
#coding: utf-8

#import des modules
#lecture des json
import json 
#affichage des dictionnaires avec idententation
import pprint
pp = pprint.PrettyPrinter(indent=4, )
#librairie des graphes
import networkx as nx
#affichage des graphs
import matplotlib
import matplotlib.pyplot as plt
#calcul
import numpy as np
from itertools import combinations
#%matplotlib inline

# Toutes les  versions de l'article 9

Nous allons charger les informations contenues dans le fichier article 9:
* la version originale (qui correspond aux statistiques finales de votes sur l'article)
* les versions complémentaires (soit les 108 modifications additionnelles proposées au vote)
selon la même nomenclature

#### Une petite explication sur la génération des slugs
Ils servent de clés au dictionnaire de versions:
Notre but étant de rappatrier les votes de chaque versions  pour chaque participant
il faut identifier de quelle version il s'agit.
Ici pour faciliter le travail et comme on peut voir plus haut
la version originale correspond au nom de générique de l'article

Recapitulons:

L'url de la version originale se présente sous cette forme:
```
urlV_0 = "http://www.republique-numerique.fr/projects/projet-de-loi-numerique/consultation/consultation/opinions/section-2-travaux-de-recherche-et-de-statistique/article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics"
```
L'url des versions se présentent toujours sous cette formes
```
urlV_add = "http://www.republique-numerique.fr/projects/projet-de-loi-numerique/consultation/consultation/opinions/section-2-travaux-de-recherche-et-de-statistique/article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics/versions/mise-a-disposition-systematique-des-resultats-de-la-recherche-financee-par-des-fonds-publics"
```

Le slug (raccourci) est produit de la manière suivante:
* on découpe les urls en morceaux a partir de "/" et on prend le dernier element de la liste de la manière suivante
```
liste_element_url = url.split("/")
slug = liste_element_url[-1]
```
    * vérifions pour la version originale:
```
liste_element_url = urlV_O.split("/")
print(liste_element _url)
slug = liste_element_url[-1]
print(slug)
```
On est censé avoir:
```
>>> ['http:', '', 'www.republique-numerique.fr', 'projects', 'projet-de-loi-numerique', 'consultation', 'consultation', 'opinions', 'section-2-travaux-de-recherche-et-de-statistique', 'article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics']
>>>'article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics'
```
    * vérifions pour une autre version:
```
liste_element_url = urlV_add.split("/")
print(liste_element _url)
slug = liste_element_url[-1]
print(slug)
```
On est censé avoir:
```
>>> ['http:', '', 'www.republique-numerique.fr', 'projects', 'projet-de-loi-numerique', 'consultation', 'consultation', 'opinions', 'section-2-travaux-de-recherche-et-de-statistique', 'article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics', 'versions', 'mise-a-disposition-systematique-des-resultats-de-la-recherche-financee-par-des-fonds-publics']
>>>'mise-a-disposition-systematique-des-resultats-de-la-recherche-financee-par-des-fonds-publics'
```

In [11]:
def load_versions(art_file="article9.json"):
    '''charger l'ensemble des versions de l'article'''
    versions_d = {}
    with open(art_file, "r") as f:
        article9 = json.load(f)
    
    #Pour rappel
    #print article9.keys()
    versions_d[article9["article_link"].split("/")[-1]] = {
                "id": 0,
                "date": article9["created_at"],
                "link": article9["article_link"],
                "slug": article9["article_link"].split("/")[-1],
                "title": article9["article_link"].split("/")[-1].replace("-", " "),
                "text": article9['body'],
                "author": article9["author"],
                "votes":[], 
                "arguments":article9["arguments"], 
                "votes_arguments": [],
                "sources":article9["sources"],
                "votes_sources": [],
                "total_votes": article9["votes_total"],
                #les décomptes de votes sur les arguments ne sont pas disponible dans l'article9
                  # seulement le nombre d'arguments
                "total_arguments_votes": article9["arguments_count"]
               }
    
    versions = article9["versions"]
    
    #on construit une liste de versions
    for i,v in enumerate(versions):
        #pour rappel
        #print v.keys()
        versions_d[v["slug"]] = {"date":v["created_at"],
                "id": i+1,
                "link":v["link"],
                #"slug":v["slug"],
                "title":v["title"],
                "text": v['comment'],
                "author": v["author"],
                "votes":[], 
                "arguments":[], 
                "votes_arguments": [],
                "sources":[],
                "votes_sources": [],
                "total_votes":v["votes_total"],
              #ici il s'agit bien du nombre de votes sur un argument
                "total_arguments_votes": v["arguments_count"]
             }
    print(len(versions_d)), "versions"
    return versions_d

# Les participants
Nous allons maintenant récupérer l'ensemble de la liste des participants au *projet de loi*

In [14]:
def load_participants():
    #charger les votes des participants
    with open("participants.json", "r") as f:
        data = json.load(f)
        participants = data["participants"]
    
    participants_d = {}
    for part in participants:
        for k,v in part.items():
            
            participants_d[k] = v["votes"]
        
    print (len(participants_d), "participants sur l'ensemble des articles")
    return participants_d

### Les electeurs de l'article 9

Nous allons dans `update_version`
mettre à jour les version en ajoutant les votes de chaque particpants
Mais comme les votes sont organisés par electeurs
nous allons regarder chaque electeur:
    - verifier qu'il a voté pour une des versions de l'article 9
    identifiable par son nom générique 
```
art = "article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics"
``` 
    - vérifier pour quelle version il a voté:
    identifiable par le slug (cf. plus haut)
    - vérifier ensuite s'il s'agit d'un vote sur une source argument ou la version elle même
    identifiable par #source-<id> ou #arg-<id> ou s'il n'en a pas il s'agit de la version elle même
    

In [16]:
def update_versions(participants, versions_d, art="article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics"):
    #pour chaque participant
    for user, votes in participants.items():
        #pour chaque vote du participants
        for vote in votes:            
            link = vote["link"]
            #a-t-il voté pour notre article?
            if art in link:
                date =  vote["date"]
                opinion = vote["opinion"]
                #pour quelle version?
                slug = link.split("/")[-1]
                
                #est ce qu'on peut découper avec #?
                # = y a t'il une info complémentaire du type #arg #source
                try:
                    #oui: c'est un vote sur un arg ou une source
                    slug, tid = slug.split("#") 
                    #exemple:
                    
                    if "arg-" in tid:
                        #dans notre dictionnaire de référence sur les versions
                        #c'est un argument on stocke dans les votes pour les arguments
                        versions_d[slug]["votes_arguments"].append({"electeur":user,"vote": opinion, "date":date, "id":tid})
                    elif "source-" in tid:
                        #c'est une sources on stocke dans les votes pour les sources
                        versions_d[slug]["votes_sources"].append({"electeur":user,"vote": opinion, "date":date, "id":tid})
                    else:
                        #au cas ou il y aurait un #autrechose mais ne s'applique pas ici
                        print slug
                except:
                    #non: c'est donc un vote simple sur une version (originale ou autre)
                    versions_d[slug]["votes"].append({"electeur":user,"vote": opinion, "date":date})
    return versions_d
    

Ici il s'agit d'une simple fonction pour faciliter les statistiques 
et pour ne garder que ce qui nous interesse on verra ici que on ne s'intéresse qu'aux votes sur les versions

Petit rappel de repartition des votes par types:
    * votes sur les versions
    * votes sur les arguments de chaque versions
    * votes sur les sources de chaque versions

Rangés dans des listes qui contiennent la date, l'electeur, le vote et son id unique
mis à plat dans `load_votes()` dans des listes de votes.

In [17]:
def load_votes(participants,art ="article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics"):
    #charger les votes des electeurs de l'article 9
    votes_v = []
    votes_args = []
    votes_src = []
    for user, votes in participants.items():
        for vote in votes:
            link = vote["link"]
            if art in link:
                date =  vote["date"]
                opinion = vote["opinion"]
                slug = link.split("/")[-1]
                try:
                    slug, tid = slug.split("#") 
                    if "arg-" in tid:
                        votes_args.append({"electeur":user,"vote": opinion, "slug":slug, "date":date, "id":tid})
                    elif "source-" in tid:
                        votes_src.append({"electeur":user,"vote": opinion, "slug":slug, "date":date, "id":tid})
                    else:
                        print slug
                except:
                    votes_v.append({"electeur":user,"vote": opinion, "slug":slug, "date":date})
            
    print len(votes_v),"votes", len(votes_args),"votes sur les arg", len(votes_src), "votes sur les sources"
    return votes_v, votes_args, votes_src
    

# Filter top_electeur
Premier objectif : calculer combien de fois un electeur a voté pour l'article (c'est à dire toutes les versions)
Deuxième objectif: qui sont les electeurs les plus actifs? 
Si on analyse la répartition des electeurs / X le nombre de votes, on se rend compte comme la plupart des datasets que 80% des electeurs ont voté pour moins de 20 % des votes
>> Dans l'ideal générer l'histogramme du nombre de vote par electeur a montrer pour justifier le seuil

Ici on filtre les electeurs les plus actifs: soit le SEUIL dit: 
l'electeur doit avoir voté au moins X fois

In [34]:
def filter_top_electeurs(votes, SEUIL):
    from collections import Counter, defaultdict
    #ici je crée la liste des electeurs uniques qui ont votés
    electeur_names = list(set([v["electeur"] for v in votes]))
    #ici je crée un dictionnaire pour chaque electeur qui contient une liste vide
    electeurs_d = defaultdict.fromkeys(electeur_names, [])
    
    #On déroule la liste des votes 
    for vote in votes:
        # chaque vote est un dictionnaire
        #qui contient le nom de l'electeur + ...
        # vote = {"electeur": "cdequatrebarbes", "opinion":1, "date": XXX, "slug": XXX, "id":5}
        #pour chaque electeur on lui ajoute son vote (ici un type de vote choisi (cf.votes))
        electeurs_d[vote["electeur"]].append(vote)
    
    #ici on compte simplement le nombre de nom dans la liste 
    f = Counter([data["electeur"] for data in votes])
    nb_votes_top_users = 0
    #f["vincentreverdy"] x fois dans la liste
    #ici ce qu'on veut c'est le 
    for name,nb_votes in f.items():
        if nb_votes < SEUIL:
            del electeurs_d[name]
            nb_votes_top_users +=nb_votes
    #Quelques stats pour justifier le seuil de participation
    nb_top_users =  len(electeurs_d)
    #on a aussi nb_votes_top_users
    nb_votes_total = len(votes)
    
    nb_total_users = len(electeur_names)
    
    
    print "TOTAL ==============="
    print "Nb Total de votes sur les versions", str(nb_votes_total)
    print "Nb Total d'electeurs sur les versions", str(nb_total_users)
    print "TOP USERS ==============="
    print "Nb d'electeur ayant votés", SEUIL, "fois:", nb_top_users
    print "Nb de votes des electeurs ayant votés", SEUIL, "fois:", nb_votes_top_users
    print "POURCENTAGE ==============="
    print "Ces electeurs ayant votés au moins", SEUIL, "fois\n représentent:"
    part_users = float(float(nb_top_users)/nb_total_users)*100
    part_votes = (float(nb_votes_top_users)/nb_votes_total)*100
    print "- % electeurs:", part_users
    print "- % votes:",part_votes
    return electeurs_d
    

In [ ]:
def calc_similarity(electeurs, versions_d, SEUIL=0):
    
    from itertools import combinations
    votes_by_version = {}
    for v, k in versions_d.items():
        votes_by_version[v] = {e["electeur"]:e["vote"] for e in k["votes"]}
        
    similarity_score = {}
    for couple in combinations(electeurs.keys(), 2):
        userA, userB = couple
        similarity_score[couple] = 0
        for k,v in votes_by_version.items():
            if userA in v.keys() and userB in v.keys():
                if v[userA] == v[userB]:
                    similarity_score[couple] += 1
    return {k:v for k,v in similarity_score.items() if v > SEUIL}
        

In [ ]:
def build_graph(simil, top_users):
    g = nx.Graph()
    pos = nx.spring_layout(g)    
    for k in simil.keys():
        g.add_nodes_from(k[0], weight=len(top_users[k[0]]))
        g.add_nodes_from(k[1], weight=len(top_users[k[1]]))
    #nodesize=[len(top_users[v])*1000 for v in g.nodes()]
    #nx.draw_networkx_nodes(g, pos, nodelist=g.nodes(), node_size=, alpha=1, label=True)
    for couple, score in simil.items():
        userA, userB = couple
        g.add_edge(userA, userB, weight=score)
        
    for node in nx.isolates(g):
        g.remove_node(node)
    nx.write_gexf(g, "electeurs_VF_SEUIL_5_5.gexf")
    nx.draw(g)
    plt.show()
    return

In [ ]:
def build_histogram(versions_nb):
    import pylab
    N = len(versions_d)
    votes_infos = [v["total_votes"]  for v in versions_nb.values()]
    
    args_infos = [v["total_arguments_votes"]  for v in versions_nb.values()]
    plt.title("Historique des votes et votes d'arguments par version")
    plt.ylabel('Nb votes')
    #pylab.xlim([0,108])
    plt.plot(votes_infos,color='r')
    plt.plot(args_infos,color='b')

    plt.show()
    plt.title("historique des votes des arguments par version")
    plt.ylabel('Nb votes sur les arguments')
    pylab.xlim([0,len(versions_nb)])
    plt.plot(args_infos,color='b')
    plt.show()
    return

In [ ]:
def build_distrib(freq):
    '''build repartition votes'''
    N = len(freq)
    plt.title("Nombre de votes par auteurs")
    plt.plot(sorted(freq.values()),color='r')
    plt.show()
    plt.title("historique des votes des arguments par version")
    plt.ylabel('Nb votes sur les arguments')
    pylab.xlim([0,N])
    pylab.ylim([1,50])
    plt.show()
    return 

In [40]:
if __name__ == "__main__":
    #versions_d = build_versions()
    #votes, electeurs_d = load_electeurs()
    vs = load_versions()
    part = load_participants()
    vs = update_versions(part, vs)
    votes,vvargs, vsources= load_votes(part)
    #SEUIL
    #Qui on considère comme top_user
    #A partir de combien de votes on peut dire qu'ils ont un comportement
    #similaire
    SEUIL = (5, 3)
    top_electeurs = filter_top_electeurs(votes, int(SEUIL[0]))
    #similar_couple = calc_similarity(top_electeurs, vs, SEUIL[1])
    #graph = build_graph(similar_couple, top_electeurs)

109 versions
(21329, "participants sur l'ensemble des articles")
12851 votes 2522 votes sur les arg 174 votes sur les sources
TOTAL ===============
Nb Total de votes sur les versions 174
Nb Total d'electeurs sur les versions 46
TOP USERS ===============
Nb d'electeur ayant votés 5 fois: 8
Nb de votes des electeurs ayant votés 5 fois: 87
POURCENTAGE ===============
Ces electeurs ayant votés au moins 5 fois
 représentent:
- % electeurs: 17.3913043478
- % votes: 50.0
